Here are the functions we will use in this notebook.

In [ ]:
from numpy import abs, argmax, array, average, ceil, exp, floor, real, vstack, where
from numpy.fft import fft, fftfreq, ifft

from scipy.io.wavfile import read as read_wav
from scipy.io.wavfile import write as write_wav

from matplotlib import pyplot as plort

First, we read in the data. For this, we can simply use the `scipy.io.wavfile` module. We will need both the sample rate and the raw data. Add code to do this, and run it.

Next, we will take the Fourier transform of the entire waveform. This can be plotted as a line plot of amplitude versus frequency. We specify `norm='ortho'` for the fft, which makes it more normalized than a default fft. The `fftfreq` function will allow you to determine the frequencies for each bin, which simplifies the x-axis of the plot.

In [ ]:
fft_full = # ADD FFT FUNCTION CALL HERE
fft_bins = fftfreq(data.size, 1.0 / float(sample_rate))

plort.plot(fft_bins, abs(fft_full))
plort.xlabel('Frequency ($\\mathrm{Hz}$)')
plort.ylabel('Amplitude')
plort.show()

Note the symmetry: the FFT is always symmetric for real data. We can thus discard up to half the data, if we care to do so.

Frequencies are most often usefully viewed on a log-log plot. In audio, humans are logarithmically sensitive to "loudness" and logarithmically sensitive to frequency, so this works quite well. We also do this in electronics fairly often, although that is mostly a matter of convenience when working with signals of variable magnitude. Doing this, we can clearly see that the original signal had five tones. However, we have lost the temporal information: we can not see if this was a sequence or combination of the tones.

The `loglog` function allows us to make these plots.

In [ ]:
# ADD PLOT COMMAND HERE
plort.xlabel('Frequency ($\\mathrm{Hz}$)')
plort.ylabel('Amplitude')
plort.show()

It is more interesting to see how the frequency varies over time. This is known as the spectrogram (most audio manipulation software will also show you this).
To create a spectrogram, we will divide the data into a set of shorter waveforms. Then, we can take the FFT of each of the waveforms. Finally, we plot this in 2D (`pcolormesh` is good for this). A Fourier transform will contain frequencies up to half the sampling frequency (which is call Nyquist). Our waveform was generated at $44100\,\mathrm{Hz}$. This is a fairly common sampling rate for audio, as it ensures that Nyquist is slightly beyond human hearing. In our case, all of our frequencies are much lower. A log plot would work well here, but that is somewhat tricky to get working with 2D plotting, so we recommend simply truncating at 100 frequencies.

Let the length of each bin be 0.05 seconds. Remember that since the length of the data has changed, we need to recalculate the bins.

In [ ]:
spectrogram_bin = 0.05
spectrogram_sample_bin = # SET TO THE NUMBER OF SAMPLES IN EACH BIN
spectrogram_bin_count =  # SET TO THE NUMBER OF BINS
fft_bins = # ADD CODE TO DETERMINE BINS FOR EACH FFT

ffts = []

for i in range(0, spectrogram_bin_count):
    slice_fft = fft(data[i * spectrogram_sample_bin:(i+1) * spectrogram_sample_bin], norm='ortho')
    ffts.append(slice_fft)

# Arrange properly and truncate to the first 100 frequencies
ffts = vstack(ffts)
ffts_positive = abs(ffts[:,0:100].transpose())

spectrogram_bin_times = # CALCULATE THE TIME INDEX OF EACH BIN

plort.pcolormesh(spectrogram_bin_times, fft_bins[0:100],
                 ffts_positive, shading='nearest', edgecolors='face')
plort.xlabel('Time ($\mathrm{s}$)')
plort.ylabel('Frequency ($\mathrm{Hz}$)')
plort.show()

We can also plot the frequency of the peak value over time. A more sophisticated analysis might find all peaks, then only consider prominent ones.

The function `argmax` lets us find the index of the peak.

In [ ]:
# ADD PLOT COMMAND HERE
plort.xlabel('Time ($\\mathrm{s}$)')
plort.ylabel('Peak Frequency ($\\mathrm{Hz}$)')
plort.show()

Another useful use of the Fourier transform is to track the amplitude of a small signal in the noise. Load the second signal file. If you listen to the file, you will not hear much besides white noise. Plotting this file suggests there is a slight distortion, but it is fairly unclear.

In [ ]:
sample_rate, data = read_wav('test_fft_signal_2.wav')

plort.plot(data)
plort.show()

If we plot the spectrogram, we can see that there is a time varying signal at 440Hz.

In [ ]:
# ADD SPECTROGRAM CODE

We can also plot the average of bins near 440Hz to see the amplitude over time.

You can use the `where` function to find the indicies of the bins we are interested in.

In [ ]:
target_bins = # SET TO BINS WE WANT TO LOOK AT
plort.plot(spectrogram_bin_times, average(abs(ffts[:,target_bins]), axis=2))
plort.show()

By filtering an FFT, we can isolate signals at certain frequencies. The tone is at 440Hz. We will use a Gaussian for the filtering. The Gaussian is defined by
$$
f(x) = e^{- (x - x_0)^2 / \sigma^2}
$$
where $x_0$ is the center and $\sigma$ is related to the width (depending on your definition of width, there may be various factors of $2$).
A spectrogram of the filtered data will likely be illuminating.

In [ ]:
def gaussian_filter(x, center_freq, width):
    return 0 # DEFINE A GAUSSIAN

filter = # CONSTRUCT THE FILTER (a list comprehension will work well)

plort.plot(fft_bins, filter)
plort.show()

filtered_ffts = ffts * filter
filtered_ffts_positive = abs(filtered_ffts[:,0:100]).transpose()

# Display the filtered data
plort.pcolormesh(spectrogram_bin_times, fft_bins[0:100],
                 filtered_ffts_positive, shading='nearest', edgecolors='face')
plort.show()

As you can see, most of the noise has been eliminated. We can then turn this back into an audio signal and save it. Note: you can and should confirm that the imaginary part of the signal is small.

To turn it back into an audio signal, we need to use the `ifft` function on each slide of the filtered fft stack.

In [ ]:
filtered_data = []

for filtered_fft in filtered_ffts:
    # COMPUTE AND APPEND THE INVERSE OF THE FILTERED FFT HERE

filtered_data = array(filtered_data, dtype='float32').reshape(len(filtered_data) * len(fft_bins))
write_wav('test_fft_signal_2_filtered_1.wav', sample_rate, filtered_data)

There are some complications to use of the FFT in practice that we have glossed over slightly here. The most relevant to what you will hear is related to "windowing". The FFT assumes that the signal repeats exactly. However-especially with our short bins-this is not really true. This also leads to a slight jump at the beginning and end of each bin after applying the filter, which causes unpleasant distortions. This can be corrected with proper use of windows, but that is beyond the scope of this overview. In addition, the precision of the FFT is limited by the length of the data. Intuitively, if you try to compare two sine waves of roughly the same frequency, the difference between them will not really be apparent until after they have completed a few cycles. It turns out that in practice, the FFT can only distinguish frequencies that are separated by more than one over the total length of the sample. Thus, we tend to get a "broadening" of the signal (which you can see as the vertical extent of the tones in the spectrogram).